In [36]:
import asyncio
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_core.model_context import BufferedChatCompletionContext

async def simulate_conversation():
    # 메모리 초기화
    chat_memory = ListMemory(name="chat_history")

    # 유저 첫 질문
    user_input1 = "안녕! 오늘 날씨 어때?"
    print(f"\n👤 사용자: {user_input1}")
    await chat_memory.add(MemoryContent(content=f"User: {user_input1}", mime_type=MemoryMimeType.TEXT))

    # 에이전트 첫 응답
    assistant_reply1 = "안녕하세요! 오늘은 맑고 화창한 날씨입니다."
    print(f"🤖 어시스턴트: {assistant_reply1}")
    await chat_memory.add(MemoryContent(content=f"Assistant: {assistant_reply1}", mime_type=MemoryMimeType.TEXT))

    # 유저 두 번째 질문
    user_input2 = "그럼 뭐 입는 게 좋을까?"
    print(f"\n👤 사용자: {user_input2}")
    await chat_memory.add(MemoryContent(content=f"User: {user_input2}", mime_type=MemoryMimeType.TEXT))

    # 에이전트 두 번째 응답
    assistant_reply2 = "가벼운 셔츠나 반팔에 청바지 정도가 좋겠네요!"
    print(f"🤖 어시스턴트: {assistant_reply2}")
    await chat_memory.add(MemoryContent(content=f"Assistant: {assistant_reply2}", mime_type=MemoryMimeType.TEXT))

    # 모델 컨텍스트에 대화 내역 업데이트
    model_context = BufferedChatCompletionContext(buffer_size=1)

    await chat_memory.update_context(model_context) #전체반환

    # 현재 컨텍스트 메시지 출력
    print("\n🧠 모델이 참고할 대화 컨텍스트:")
    context_messages = await model_context.get_messages()
    for i, msg in enumerate(context_messages):
        print(f"  {i+1}. {msg.content}")

    # 마지막으로 메모리 내부를 확인
    print("\n📦 저장된 메모리 내역:")
    all_memories = await chat_memory.query()
    for i, m in enumerate(all_memories.results):
        print(f"  {i+1}. {m.content}")

    await chat_memory.close()

# 실행
await simulate_conversation()



👤 사용자: 안녕! 오늘 날씨 어때?
🤖 어시스턴트: 안녕하세요! 오늘은 맑고 화창한 날씨입니다.

👤 사용자: 그럼 뭐 입는 게 좋을까?
🤖 어시스턴트: 가벼운 셔츠나 반팔에 청바지 정도가 좋겠네요!

🧠 모델이 참고할 대화 컨텍스트:
  1. 
Relevant memory content (in chronological order):
1. User: 안녕! 오늘 날씨 어때?
2. Assistant: 안녕하세요! 오늘은 맑고 화창한 날씨입니다.
3. User: 그럼 뭐 입는 게 좋을까?
4. Assistant: 가벼운 셔츠나 반팔에 청바지 정도가 좋겠네요!


📦 저장된 메모리 내역:
  1. User: 안녕! 오늘 날씨 어때?
  2. Assistant: 안녕하세요! 오늘은 맑고 화창한 날씨입니다.
  3. User: 그럼 뭐 입는 게 좋을까?
  4. Assistant: 가벼운 셔츠나 반팔에 청바지 정도가 좋겠네요!


In [ ]:
#장기 , 단기 메모리 구현
from autogen_core.memory import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig

# 벡터 DB 기반 장기 메모리 초기화
long_term_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="user_knowledge",
        persistence_path="/path/to/db",
        k=3  # 상위 3개 결과 반환
    )
)

# 중요 정보 저장
await long_term_memory.add(
    MemoryContent(
        content="사용자는 비건 식단을 선호합니다.",
        mime_type=MemoryMimeType.TEXT,
        metadata={"category": "preferences", "importance": "high"}
    )
)

In [ ]:
# 리스트 기반 단기 메모리 초기화
short_term_memory = ListMemory(name="session_context")

# 현재 세션 정보 저장
await short_term_memory.add(
    MemoryContent(
        content="사용자는 현재 여행 계획을 논의 중입니다.",
        mime_type=MemoryMimeType.TEXT
    )
)

In [ ]:
from autogen_core.agent import AssistantAgent
from autogen_core.model_client import ModelClient

model_client = ModelClient(model_name="gpt-4o-mini")
agent = AssistantAgent(
    name="travel_assistant",
    model_client=model_client,
    memory=[long_term_memory, short_term_memory], #2개의 메모리
    system_message="사용자의 선호도를 고려하여 여행 계획을 도와주세요."
)

In [ ]:
# 리스트 기반 단기 메모리 초기화
short_term_memory = ListMemory(name="session_context")

# 현재 세션 정보 저장
await short_term_memory.add(
    MemoryContent(
        content="사용자는 현재 여행 계획을 논의 중입니다.",
        mime_type=MemoryMimeType.TEXT
    )
)

In [ ]:
# 현재 세션 상태 저장
agent_state = await agent.save_state()
with open("user123_session.json", "w") as f:
    json.dump(agent_state, f)

# 나중에 세션 복원
with open("user123_session.json", "r") as f:
    saved_state = json.load(f)
await agent.load_state(saved_state)